랜덤 포레스트, 엑스트라 트리, SVM 같은 여러 종류 분류기 훈련. </br>
검증 세트에서 간접/직접 투표 방법으로 앙상블 연결.
테스트 세트로 확인. </br>
08팀 이헌성, 10팀 박경준

<출처> </br>
1. https://childult-programmer.tistory.com/71
2. https://mldlcvmjw.tistory.com/216
3. https://diffrentedcon.tistory.com/166
4. https://guru.tistory.com/36

In [16]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn import datasets
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC

In [17]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False)

In [18]:
# train에 50,000개 샘플, test에 10,000개 샘플. (검증에 10,000개 샘플)
from sklearn.model_selection import train_test_split

X_train_val,X_test,y_train_val,y_test=train_test_split(
    mnist.data,mnist.target,test_size=10000,random_state=42
)

X_train,X_val,y_train,y_val=train_test_split(
    X_train_val,y_train_val,test_size=10000,random_state=42
)

In [19]:
random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm_clf = LinearSVC(max_iter=100, tol=20, random_state=42)

In [20]:
estimators = [random_forest_clf, extra_trees_clf, svm_clf]
for estimator in estimators:
  print('Training: ', estimator)
  estimator.fit(X_train, y_train)

Training:  RandomForestClassifier(random_state=42)
Training:  ExtraTreesClassifier(random_state=42)
Training:  LinearSVC(max_iter=100, random_state=42, tol=20)


In [21]:
for estimator in estimators:
  print(estimator.score(X_val, y_val))

0.9692
0.9715
0.0997


In [22]:
# 직접 투표
from sklearn.ensemble import VotingClassifier

named_estimators = [
    ("rf", random_forest_clf),
    ("et", extra_trees_clf),
    ("svm", svm_clf),
]

voting_clf = VotingClassifier(named_estimators)

voting_clf.fit(X_train, y_train)

voting_clf.score(X_val, y_val)

0.9642

In [23]:
# svm 별로여서 제거
voting_clf.set_params(svm=None)
del voting_clf.estimators_[2]
voting_clf.voting='soft'
voting_clf.score(X_val,y_val)

0.9719

In [24]:
voting_clf.score(X_test,y_test)

0.9681